In [1]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [43]:
data_path = "../../data"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [107]:
a = pd.read_csv("./checkcheck.csv")
a = list(a['question'])
query = a

In [9]:
pickle_name = f"sparse_embedding100.bin"
tfidfv_name = f"tfidv100.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding100 = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv100 = pickle.load(file)
    print("Embedding pickle load.")

Embedding pickle load.


In [75]:
query_vec100 = tfidfv100.transform(query)

In [78]:
query_cut = tfidfv100.transform(query)

In [82]:
query_cut_last_space = tfidfv100.transform(query)


In [111]:
query_cut_josa_last_space = tfidfv100.transform(query)

In [108]:
for i,v in enumerate(query):
    if "는?" in v or "은?" in v:
        query[i] = query[i][:-2]
    else:
        query[i] = query[i][:-3]

In [109]:
for i, v in enumerate(query):
    for ii in range(len(v)-1,0,-1):
        if query[i][ii] == " ":
            query[i] = query[i][0:ii]
            break

In [25]:
a = pd.read_csv("./checkcheck.csv")
a= list(a['original_context'])
tmp = []
for v in a:
    for ii,vv in enumerate(contexts):
        if v == vv:
            tmp.append(ii)

In [76]:
    k = 1
    result = query_vec100 * p_embedding100.T
    #
    for k in range(0, 100, 5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  74.16666666666667
top  10  acc:  82.5
top  15  acc:  85.83333333333333
top  20  acc:  87.08333333333333
top  25  acc:  88.75
top  30  acc:  89.16666666666667
top  35  acc:  91.25
top  40  acc:  91.66666666666666
top  45  acc:  92.5
top  50  acc:  92.5
top  55  acc:  92.91666666666667
top  60  acc:  93.33333333333333
top  65  acc:  93.33333333333333
top  70  acc:  93.33333333333333
top  75  acc:  94.16666666666667
top  80  acc:  94.16666666666667
top  85  acc:  95.0
top  90  acc:  95.41666666666667
top  95  acc:  96.25


In [64]:
for i, v in enumerate(query):
    for ii in range(len(v)-1,0,-1):
        if query[i][ii] == " ":
            query[i] = query[i][0:ii]
            break
        

In [44]:
check = []
for i in index:
    context = ""
    for ii in doc_indices[i]:
        context += contexts[ii]+"\n"+"@@@@@@@"+"\n"
    check.append(context)

In [56]:
answer = []
question = []
for i in index:
    answer.append(contexts[tmp[i]])
    question.append(query[i])

In [61]:
go = pd.DataFrame({"question":question,"predict":check, "answer":answer}) 

In [62]:
go.to_csv("./전처리체크.csv")

In [79]:
    result = query_cut * p_embedding100.T
    #
    for k in range(0, 100, 5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  75.83333333333333
top  10  acc:  83.75
top  15  acc:  86.66666666666667
top  20  acc:  87.91666666666667
top  25  acc:  88.75
top  30  acc:  90.41666666666667
top  35  acc:  90.41666666666667
top  40  acc:  92.08333333333333
top  45  acc:  92.5
top  50  acc:  92.91666666666667
top  55  acc:  94.16666666666667
top  60  acc:  94.58333333333333
top  65  acc:  95.0
top  70  acc:  95.0
top  75  acc:  95.0
top  80  acc:  95.83333333333334
top  85  acc:  95.83333333333334
top  90  acc:  96.25
top  95  acc:  96.25


In [83]:
    result = query_cut_last_space * p_embedding100.T
    #
    for k in range(0, 100, 5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  76.66666666666667
top  10  acc:  83.75
top  15  acc:  86.25
top  20  acc:  87.08333333333333
top  25  acc:  87.91666666666667
top  30  acc:  89.58333333333334
top  35  acc:  89.58333333333334
top  40  acc:  90.41666666666667
top  45  acc:  91.25
top  50  acc:  92.08333333333333
top  55  acc:  93.33333333333333
top  60  acc:  94.16666666666667
top  65  acc:  94.58333333333333
top  70  acc:  94.58333333333333
top  75  acc:  95.41666666666667
top  80  acc:  95.83333333333334
top  85  acc:  95.83333333333334
top  90  acc:  95.83333333333334
top  95  acc:  96.25


In [110]:
for i in range(len(query)):
    query[i] = query[i][0:-1]

In [112]:
    result = query_cut_josa_last_space * p_embedding100.T
    #
    for k in range(0, 100, 5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  77.08333333333334
top  10  acc:  83.75
top  15  acc:  87.08333333333333
top  20  acc:  87.5
top  25  acc:  88.33333333333333
top  30  acc:  89.58333333333334
top  35  acc:  90.0
top  40  acc:  90.83333333333333
top  45  acc:  92.08333333333333
top  50  acc:  92.5
top  55  acc:  94.16666666666667
top  60  acc:  94.58333333333333
top  65  acc:  94.58333333333333
top  70  acc:  95.0
top  75  acc:  95.41666666666667
top  80  acc:  95.83333333333334
top  85  acc:  95.83333333333334
top  90  acc:  95.83333333333334
top  95  acc:  95.83333333333334


In [95]:
query

['처음으로 부실 경영인에 대한 보상 선고를 받은',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된',
 '로타이르가 백조를 구하기 위해 사용한',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을',
 '루이 14세의 왕비 마리아 테래사는 어느 나라',
 '헤자즈 왕국이 실존했던 것은',
 '버드 교장이 5월의 여왕의 대안으로 제시한',
 "인형사'를 만들어낸 것으로 추측되는 사업의",
 '멘데스가 요원들을 구하기 위해 간 도시는',
 '교과부의 행동에 화가나 여러명이 사직한 기구의',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은',
 '피어슨이 다시 의회를 해산했던',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의',
 '이흥구의 사법시험 이야기를 기사로 작성한',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던',
 '박지훈은 1라운드에서 몇 순위를',
 '데메카론에는 무엇을 풍자하는 이야기가',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의',
 '설리반이 불만을 표시한 대상은',
 '베소스는 어디서',
 '진전사의 명칭이 드러나는 데 영향을 준',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의',
 '박제된 북극곰이 사망한',
 '문법 측면에서 더 보수적인 포르투갈어',
 '로스 수장이 살해한 사람은 어느 당',
 '조경숙왕의 아들인 요자의 친어머니는',
 '오래플린과 부스의 마지막 계획에 따르면 그들은 어디서 링컨을 납치하려고',
 '김득황이 친일파로 취급되었던 것은 무슨 경력',
 '레닌이 출간한 책 중 농민의 자발적 참여에 대한 내용이 포함되어있는',
 '신란의 동반자가 죽었다고 전해지는',
 '칭자오의 머리가 엄청난 위력을 발휘